# How do socioeconomic factors affect crime rates in Seattle?
## Chantria Im, Ellie Velez, and Nawaf Osman

In [1]:
# Import libraries
import pandas as pd
import requests
import re
import dill # Notebook caching
import time

In [2]:
# Load datasets
crime_df = pd.read_csv('./data/SPD_Crime_Data__2008-Present_20240226.csv')
poverty_df = pd.read_csv('./data/Poverty_and_Near_Poverty_Map_Full_Data_data.csv', converters={'Census Tract': str.strip})
income_df = pd.read_csv('./data/ACSST5Y2018.S1903-Data.csv')

In [3]:
# Remove rows with missing Longitude and Latitude values
crime_df = crime_df[(crime_df['Longitude'] != 0) & (crime_df['Latitude'] != 0)]

# Filter for only 2018 entries
crime_df['Report DateTime'] = pd.to_datetime(crime_df['Report DateTime'], format='%m/%d/%Y %I:%M:%S %p')
crime_df = crime_df[crime_df['Report DateTime'].dt.year == 2018]

# Add Census Tract column
crime_df['Census Tract'] = pd.Series(dtype='string')

In [4]:
# Cache the dataframe
# %store crime_df
# Reload the dataframe from cache rather than recomputing (5+ hours)
%store -r crime_df
crime_df

,Report Number,Offense ID,Offense Start DateTime,Offense End DateTime,Report DateTime,Group A B,Crime Against Category,Offense Parent Group,Offense,Offense Code,Precinct,Sector,Beat,MCPP,100 Block Address,Longitude,Latitude,Census Tract
41275,2018-487339,7644443754,12/31/2018 11:39:00 PM,NaN,2018-12-31 23:39:00,A,PROPERTY,STOLEN PROPERTY OFFENSES,Stolen Property Offenses,280,W,Q,Q1,MAGNOLIA,26XX BLOCK OF BISHOP PL W,-122.404822,47.644858,56
41276,2018-487178,7696639702,12/31/2018 07:00:00 PM,NaN,2018-12-31 23:14:00,A,PROPERTY,LARCENY-THEFT,All Other Larceny,23H,S,R,R2,CLAREMONT/RAINIER VISTA,35XX BLOCK OF RAINIER AVE S,-122.292905,47.571475,101
41277,2018-487186,7664183259,12/31/2018 09:14:00 PM,NaN,2018-12-31 22:52:00,B,SOCIETY,DRIVING UNDER THE INFLUENCE,Driving Under the Influence,90D,E,E,E2,CAPITOL HILL,14TH AVE / E UNION ST,-122.314167,47.612917,75
41278,2018-487085,7685572628,12/31/2018 03:00:00 PM,12/31/2018 03:30:00 PM,2018-12-31 22:09:00,A,PROPERTY,LARCENY-THEFT,Theft From Motor Vehicle,23F,E,C,C2,MONTLAKE/PORTAGE BAY,LAKE WASHINGTON BLVD E / E INTERLAKEN BLVD,-122.297248,47.630773,62
41279,2018-487216,7631819895,12/31/2018 09:33:00 PM,NaN,2018-12-31 21:33:00,A,PROPERTY,DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,Destruction/Damage/Vandalism of Property,290,N,J,J2,GREENWOOD,1XX BLOCK OF N 85TH ST,-122.357312,47.690607,17.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1071747,2018-336488,7663881810,09/08/2018 10:32:00 PM,NaN,2018-09-08 22:32:00,A,SOCIETY,DRUG/NARCOTIC OFFENSES,Drug/Narcotic Violations,35A,S,S,S1,NEW HOLLY,71XX BLOCK OF HOLLY PARK DR S,-122.287444,47.539097,110.01
1071748,2018-336488,7663881807,09/08/2018 10:32:00 PM,NaN,2018-09-08 22:32:00,A,PERSON,ASSAULT OFFENSES,Aggravated Assault,13A,S,S,S1,NEW HOLLY,71XX BLOCK OF HOLLY PARK DR S,-122.287444,47.539097,110.01
1080980,2018-350575,7669099329,09/19/2018 05:43:00 PM,NaN,2018-09-19 17:43:00,A,PROPERTY,BURGLARY/BREAKING&ENTERING,Burglary/Breaking & Entering,220,S,R,R1,MID BEACON HILL,40XX BLOCK OF 14TH AVE S,-122.314717,47.568641,100.01
1080981,2018-103499,7676453184,03/23/2018 01:38:00 PM,NaN,2018-03-23 14:02:00,A,SOCIETY,DRUG/NARCOTIC OFFENSES,Drug/Narcotic Violations,35A,W,M,M3,DOWNTOWN COMMERCIAL,15XX BLOCK OF 3RD AVE,-122.338247,47.610246,81
